<a href="https://colab.research.google.com/github/thierrydecae/association-coefficients/blob/main/Distance_Matrix_Association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import itertools
from itertools import combinations
import math
from math import sqrt

In [ ]:
# http://ramet.elte.hu/~podani/3-Distance,%20similarity.pdf

In [ ]:
trx=pd.DataFrame({"apple":[1,1,1,1,0,0,0],
                  "beer":[1,1,1,0,1,1,0],
                  "rice":[1,1,0,0,1,1,0],
                  "chicken":[1,0,0,0,0,0,0],
                  "mango":[0,0,0,1,0,0,1],
                  "milk":[0,0,0,0,1,1,1]})

trx

,apple,beer,chicken,mango,milk,rice
0,1,1,1,0,0,1
1,1,1,0,0,0,1
2,1,1,0,0,0,0
3,1,0,0,1,0,0
4,0,1,0,0,1,1
5,0,1,0,0,1,1
6,0,0,0,1,1,0


In [ ]:
def concord(df,var1,var2):
    global A,B,C,D
    A=len(df[(df[var1]>0) & (df[var2]>0)])
    B=len(df[(df[var1]<1) & (df[var2]>0)])
    C=len(df[(df[var1]>0) & (df[var2]<1)])
    D=len(df[(df[var1]<1) & (df[var2]<1)])
    return A,B,C,D

def yq (df,var1,var2):
    concord(df,var1,var2)
    stat=((A*D)-(B*C))/((A*D)+(B*C))
    return stat

def jc (df,var1,var2):
    concord(df,var1,var2)
    stat=A/(A+B+C)
    return stat

def eucl (df,var1,var2):
    concord(df,var1,var2)
    stat=math.sqrt(math.pow(A-C,2)+math.pow(B-D,2))
    return stat

def SokalM (df,var1,var2):
    concord(df,var1,var2)
    stat=(A+D)/(A+B+C+D)
    return stat

def man (df,var1,var2):
    concord(df,var1,var2)
    stat=(abs(C-A))+(abs(D-B))
    return stat 

def compute_matrix(indf,test):
    pair1=[]
    pair2=[]
    yqs=[]
    combs=[]
    val= None
    for combo in combinations(indf.columns.values,2):
        if test =='YQ':
            val=yq(indf,combo[0],combo[1])
        elif test == 'JC':
            val=jc(indf,combo[0],combo[1])
        elif test == 'SokaM':
            val=SokalM(indf,combo[0],combo[1])
        elif test == 'Eucl':
            val=eucl(indf,combo[0],combo[1])
        elif test == 'Man':
            val=man(indf,combo[0],combo[1])
        
        pair1.append(combo[0])
        pair2.append(combo[1])
        yqs.append(val)
#       Append the Reverse Using Previous Value
        pair1.append(combo[1])
        pair2.append(combo[0])
        yqs.append(val)
#       Append 'Diagonals' with 0
    for each in indf.columns.values:
        pair1.append(each)
        pair2.append(each)
        yqs.append(0)
#       Construct Output Matrix
    pair1=pd.DataFrame(pair1)
    pair2=pd.DataFrame(pair2)
    yqs=pd.DataFrame(yqs)
    outdf=pd.concat([pd.DataFrame(pair1),pd.DataFrame(pair2),pd.DataFrame(yqs)],axis=1)
    outdf.columns=['pair1','pair2','stat']
    outdf=outdf.pivot(index='pair1',columns='pair2',values='stat')
    return outdf
    

In [ ]:
procdf=compute_matrix(trx,'YQ')
procdf

pair2,apple,beer,chicken,mango,milk,rice
pair1,,,,,,
apple,0.000000,0.2,1.0,-0.2,-1.000000,-0.333333
beer,0.200000,0.0,1.0,-1.0,-0.200000,1.000000
chicken,1.000000,1.0,0.0,-1.0,-1.000000,1.000000
mango,-0.200000,-1.0,-1.0,0.0,0.200000,-1.000000
milk,-1.000000,-0.2,-1.0,0.2,0.000000,0.333333
rice,-0.333333,1.0,1.0,-1.0,0.333333,0.000000


In [ ]:
procdf=compute_matrix(trx,'JC')
procdf

pair2,apple,beer,chicken,mango,milk,rice
pair1,,,,,,
apple,0.000000,0.500000,0.25,0.20,0.000000,0.333333
beer,0.500000,0.000000,0.20,0.00,0.333333,0.800000
chicken,0.250000,0.200000,0.00,0.00,0.000000,0.250000
mango,0.200000,0.000000,0.00,0.00,0.250000,0.000000
milk,0.000000,0.333333,0.00,0.25,0.000000,0.400000
rice,0.333333,0.800000,0.25,0.00,0.400000,0.000000


In [ ]:
procdf=compute_matrix(trx,'SokaM')
procdf

pair2,apple,beer,chicken,mango,milk,rice
pair1,,,,,,
apple,0.000000,0.571429,0.571429,0.428571,0.000000,0.428571
beer,0.571429,0.000000,0.428571,0.000000,0.428571,0.857143
chicken,0.571429,0.428571,0.000000,0.571429,0.428571,0.571429
mango,0.428571,0.000000,0.571429,0.000000,0.571429,0.142857
milk,0.000000,0.428571,0.428571,0.571429,0.000000,0.571429
rice,0.428571,0.857143,0.571429,0.142857,0.571429,0.000000


In [ ]:
procdf=compute_matrix(trx,'Eucl')
procdf

pair2,apple,beer,chicken,mango,milk,rice
pair1,,,,,,
apple,0.000000,2.236068,3.605551,2.236068,5.0,1.000000
beer,2.236068,0.000000,3.605551,5.385165,1.0,3.605551
chicken,3.605551,3.605551,0.000000,2.236068,1.0,1.000000
mango,2.236068,5.385165,2.236068,0.000000,1.0,3.605551
milk,5.000000,1.000000,1.000000,1.000000,0.0,1.000000
rice,1.000000,3.605551,1.000000,3.605551,1.0,0.000000


In [ ]:
procdf=compute_matrix(trx,'Man')
procdf

pair2,apple,beer,chicken,mango,milk,rice
pair1,,,,,,
apple,0,3,5,3,7,1
beer,3,0,5,7,1,5
chicken,5,5,0,3,1,1
mango,3,7,3,0,1,5
milk,7,1,1,1,0,1
rice,1,5,1,5,1,0
